In [1]:
import sys

sys.path.append('../..')

import torch
import pytorch_lightning as pl
from pytorch_lightning.utilities.model_summary import ModelSummary

from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim

from pytorch_lightning.callbacks import ModelCheckpoint

import matplotlib.pyplot as plt
import numpy as np
import glob
import os
import pickle
import random
from tqdm import tqdm
from scipy.signal import resample

from IPython.display import Audio
from transformers import VideoMAEForVideoClassification
from typing import Dict, Any

from VISAudioAutoEncoder import VISVMAEModel, AudioAutoEncoderConv
from VISTorchUtils import VISDatasetV3

/home2/aa_aanegola/miniconda3/envs/cv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
aae = AudioAutoEncoderConv()
aae = aae.load_from_checkpoint('./model_weights/audioautoencoderconvsm-model-epoch=90-val_loss=0.00.ckpt')

In [7]:
vis_vmae = VISVMAEModel(aae)

In [8]:
valDataset = VISDatasetV3('/scratch/aanegola/test', sr=48000)

/scratch/aanegola/test/*.pkl


/home2/aa_aanegola/miniconda3/envs/cv/lib/python3.9/site-packages/transformers/models/videomae/feature_extraction_videomae.py:28: FutureWarning: The class VideoMAEFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use VideoMAEImageProcessor instead.
  warnings.warn(


In [9]:
BATCH_SIZE = 16
NUM_WORKERS = 1

In [10]:
valDataLoader = DataLoader(valDataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

In [11]:
logger = pl.loggers.TensorBoardLogger('tb_logs', name='audio_autoencoder')
checkpoint_callback = ModelCheckpoint(
    save_top_k=1,
    monitor="val_loss",
    mode="min",
    dirpath="model_weights",
    filename="audioautoencoderconv-model-{epoch:02d}-{val_loss:.2f}",
)
trainer = pl.Trainer(accelerator='gpu', devices=1,
                     max_epochs=10, logger=logger,
                     callbacks=[checkpoint_callback])
torch.set_float32_matmul_precision('medium')

/home2/aa_aanegola/miniconda3/envs/cv/lib/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home2/aa_aanegola/miniconda3/envs/cv/lib/python3.9/ ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [12]:
trainer.fit(vis_vmae, valDataLoader, valDataLoader)
trainer.save_checkpoint('model_weights/final-autoencoderconv-model.ckpt')

/home2/aa_aanegola/miniconda3/envs/cv/lib/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home2/aa_aanegola/miniconda3/envs/cv/lib/python3.9/ ...
  rank_zero_warn(
/home2/aa_aanegola/miniconda3/envs/cv/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:612: UserWarning: Checkpoint directory /home2/aa_aanegola/visually-indicated-sounds/src/experiments/AudioAutoEncoder/model_weights exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type                           | Params
--------------------------------------------------------------------
0 | featureExtractor | VideoMAEForVideoClassification | 86.2 M
1 | mlp              |

Sanity Checking: 0it [00:00, ?it/s]

/home2/aa_aanegola/miniconda3/envs/cv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Sanity Checking DataLoader 0:  50%|█████     | 1/2 [00:01<00:01,  1.51s/it]

/home2/aa_aanegola/miniconda3/envs/cv/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:76: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 16. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


/home2/aa_aanegola/miniconda3/envs/cv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0:   7%|▋         | 29/440 [00:38<08:58,  1.31s/it, v_num=5, train_loss_step=0.0763]

/home2/aa_aanegola/miniconda3/envs/cv/lib/python3.9/site-packages/pytorch_lightning/trainer/call.py:54: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
